# Low-rank approximation on $\mathcal{P}(d)$ - the space of $d$-dimensional SPD matrices

In this notebook we want to get some intuition in different approaches for computing low-rank approximations for manifold-valued signals

In [96]:
using Manifolds
using Manopt
using LinearAlgebra
using Random
using Plots
using LaTeXStrings

In [153]:
include("../../../src/decompositions/signals/naive_low_rank_approximation.jl")
include("../../../src/decompositions/signals/decoupled_curvature_corrected_low_rank_approximation.jl")
include("../../../src/decompositions/signals/exact_low_rank_approximation.jl")

include("../../../src/functions/loss_functions/curvature_corrected_loss.jl")
include("../../../src/functions/loss_functions/exact_loss.jl")

exact_loss (generic function with 2 methods)

### Load data and construct manifold ###

In [98]:
# load data
M = SymmetricPositiveDefinite(3)
d = manifold_dimension(M)
n = 100  # 100


100

In [99]:
e = 1. * Matrix(I, 3, 3)
# compute basis
Θ = get_basis(M, e, DefaultOrthonormalBasis())
#  construct data
τ = 2.  # variance
σ = .05  # variance
Xₑ = Θ.data[4]
print(Xₑ)

Random.seed!(31)
predata = [exp(M, e, sqrt(τ) * randn(1)[1] * Xₑ) for i in 1:n]

data = [exp(M, predata[i], random_tangent(M, predata[i], Val(:Gaussian), σ)) for i in 1:n]; # ∈ P(3)^n


[0.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 0.0]

In [31]:
# Export slice image
num_export = 10
asymptote_export_SPD("results/artificial1D_orig.asy", data=data[1:min(num_export,n)], scale_axes=(2,2,2)); 

### Construct low rank approximation ###

In [100]:
q = mean(M, data)
log_q_data = log.(Ref(M), Ref(q), data);  # ∈ T_q P(3)^n

In [154]:
decoupled_curvature_corrected_low_rank_approximation(M, q, data, 1; stepsize=1/10000, max_iter=20); 

[0.043589454988513694]
[0.0010403062125233344, 0.0010406362488835032, 0.0010390069952874303, 0.0010401523323877076, 0.0010393606751681163, 0.0010403383927039052, 0.0010405623195268619, 0.0010402442880790895, 0.0010404363406080629, 0.0010406011360499284, 0.001040155951630299, 0.0010405948810707382, 0.0010387034470706138, 0.0010404191335836013, 0.0010404623236762733, 0.0010406018747648042, 0.001040081720272678, 0.0010390212763687551, 0.0010399826929433404, 0.0010387174437250348, 0.001040333738558816, 0.0010406492441793489, 0.0010406384560479021, 0.0010402236561665127, 0.001040575161333663, 0.0010405967579863859, 0.001040397391846356, 0.00104059820557117, 0.0010406340333391506, 0.0010400780161924405, 0.001040653457141676, 0.0010406319497439996, 0.0010405130172421928, 0.0010403741966061688, 0.001040246051812385, 0.0010403796497134138, 0.0010406549387215963, 0.0010386317963839303, 0.0010405924333843695, 0.0010399335013326415, 0.001040466337467664, 0.001040501522150828, 0.0010404950343403936

In [34]:
# exact_low_rank_approximation(M, q, data, 1; stepsize=1/1000, max_iter=200); 

In [148]:
max_iter = 50

nR_q = []
nU = []
ccR_q = []
ccU = []
eR_q = []
eU = []
for i in 1:2
    println("#$(i) | computing naive low-rank approximation")
    nRr_q, nUr = naive_low_rank_approximation(M, q, data, i)
    push!(nR_q, nRr_q)
    push!(nU, nUr)
    println("#$(i) | computing curvature corrected low-rank approximation")
    ccRr_q, ccUr = decoupled_curvature_corrected_low_rank_approximation(M, q, data, i; stepsize=1/100000, max_iter=max_iter); 
    push!(ccR_q, ccRr_q)
    push!(ccU, ccUr)
    println("#$(i) | computing exact low-rank approximation")
    eRr_q, eUr = exact_low_rank_approximation(M, q, data, i; stepsize=1/1000, max_iter=max_iter); 
    push!(eR_q, eRr_q)
    push!(eU, eUr)
end

#1 | computing naive low-rank approximation
#1 | computing curvature corrected low-rank approximation
[0.0010403062125233344, 0.0010406362488835032, 0.0010390069952874303, 0.0010401523323877076, 0.0010393606751681163, 0.0010403383927039052, 0.0010405623195268619, 0.0010402442880790895, 0.0010404363406080629, 0.0010406011360499284, 0.001040155951630299, 0.0010405948810707382, 0.0010387034470706138, 0.0010404191335836013, 0.0010404623236762733, 0.0010406018747648042, 0.001040081720272678, 0.0010390212763687551, 0.0010399826929433404, 0.0010387174437250348, 0.001040333738558816, 0.0010406492441793489, 0.0010406384560479021, 0.0010402236561665127, 0.001040575161333663, 0.0010405967579863859, 0.001040397391846356, 0.00104059820557117, 0.0010406340333391506, 0.0010400780161924405, 0.001040653457141676, 0.0010406319497439996, 0.0010405130172421928, 0.0010403741966061688, 0.001040246051812385, 0.0010403796497134138, 0.0010406549387215963, 0.0010386317963839303, 0.0010405924333843695, 0.0010399

In [149]:
ref_distance = sum(distance.(Ref(M), Ref(q), data).^2)

naive_tangent_distances_r = zeros(d)
predicted_naive_distances_r= zeros(d)
true_naive_distances_r= zeros(d)

curvature_corrected_tangent_distances_r = zeros(d)
predicted_curvature_corrected_distances_r = zeros(d)
true_curvature_corrected_distances_r = zeros(d)

exact_tangent_distances_r = zeros(d)
exact_distances_r= zeros(d)

for rank in 1:2
    naive_log_q_data_r = Symmetric.([sum([nR_q[rank][i] * nU[rank][k,i] for i in 1:rank]) for k in 1:n])
    curvature_corrected_log_q_data_r = Symmetric.([sum([ccR_q[rank][i] * ccU[rank][k,i] for i in 1:rank]) for k in 1:n])
    exact_log_q_data_r = Symmetric.([sum([eR_q[rank][i] * eU[rank][k,i] for i in 1:rank]) for k in 1:n])
    
    # expoentiate back
    naive_data_r = exp.(Ref(M), Ref(q), naive_log_q_data_r)
    curvature_corrected_data_r = exp.(Ref(M), Ref(q), curvature_corrected_log_q_data_r)
    exact_data_r = exp.(Ref(M), Ref(q), exact_log_q_data_r)


    # compute relative tangent space error
    naive_tangent_distances_r[rank] = sum(norm.(Ref(M), Ref(q),  log_q_data - naive_log_q_data_r).^2) / ref_distance
    curvature_corrected_tangent_distances_r[rank] = sum(norm.(Ref(M), Ref(q),  log_q_data - curvature_corrected_log_q_data_r).^2) / ref_distance
    exact_tangent_distances_r[rank] = sum(norm.(Ref(M), Ref(q),  log_q_data - exact_log_q_data_r).^2) / ref_distance


    # compute relative manifold error
    predicted_naive_distances_r[rank] = curvature_corrected_loss(M, q, data, naive_log_q_data_r)
    true_naive_distances_r[rank] = exact_loss(M, q, data, naive_log_q_data_r)
    predicted_curvature_corrected_distances_r[rank] = curvature_corrected_loss(M, q, data, curvature_corrected_log_q_data_r)
    true_curvature_corrected_distances_r[rank] = exact_loss(M, q, data, curvature_corrected_log_q_data_r)
    exact_distances_r[rank] = exact_loss(M, q, data, exact_log_q_data_r)
end

In [150]:
# We want plots with (1) the lower bound error, (2) the actually uncorrected manifold error and (3) the corrected manifold error
plot(1:d, [naive_tangent_distances_r, true_naive_distances_r, true_curvature_corrected_distances_r, exact_distances_r], label = ["theoretical lower bound" "naive" "curvature corrected" "exact"], ylims=(0,1), xlims=(1,d),xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$"))
savefig("results/artificial1D_errors_by_rank.png")
plot(1:d, [naive_tangent_distances_r .+ 1e-16, true_naive_distances_r .+ 1e-16, true_curvature_corrected_distances_r .+ 1e-16, exact_distances_r .+ 1e-16], label = ["theoretical lower bound" "naive" "curvature corrected" "exact"], ylims=(1e-16,1), xlims=(1,d), xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$", :log), legend=:bottomleft)
savefig("results/artificial1D_logerrors_by_rank.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/1D/P3/results/artificial1D_logerrors_by_rank.png"

In [40]:
# It would be nice to also have a plot that tells us something about the error in predicting the manifold loss (using CCL) and the actual loss 
# (1) for the naive approach (2) for the curvature corrected approach
plot(1:d, [(predicted_naive_distances_r .+ 1e-16) ./ (true_naive_distances_r .+ 1e-16), (predicted_curvature_corrected_distances_r .+ 1e-16) ./ (true_curvature_corrected_distances_r .+ 1e-16)], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,d),xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$"))
savefig("results/artificial1D_discrepancy_by_rank.png")
plot(1:d, [(predicted_naive_distances_r .+ 1e-16) ./ (true_naive_distances_r .+ 1e-16), (predicted_curvature_corrected_distances_r .+ 1e-16) ./ (true_curvature_corrected_distances_r .+ 1e-16)], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,d), xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$", :log), legend=:bottomleft)
savefig("results/artificial1D_logdiscrepancy_by_rank.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/1D/P3/results/artificial1D_logdiscrepancy_by_rank.png"